In [43]:
import numpy as np
import scipy as sp
# import torch

In [4]:
from funcy import *

In [57]:
from functools import reduce

In [ ]:
#TODO:
# recursive filler role bindings + tree examples; 
# ...
# systematize into something more library shaped

# Local representations and familiar symbolic operations on them

Running example of a binary relation + code for creating local representations ('one hot vectors'), taken largely from https://www.cse.unsw.edu.au/~billw/cs9444/tensor-stuff/tensor-intro-04.html

`animal_food_dl` represents a binary relation between a finite set of `Animals` and a finite set of `Foods`:

In [144]:
animal_food_dl = (
    {'Animal':'rabbit', 'Food':'carrot'}, 
    {'Animal':'mouse', 'Food':'cheese'}, 
    {'Animal':'crocodile', 'Food':'student'},
    {'Animal':'rabbit', 'Food':'lettuce'},
    {'Animal':'guineapig', 'Food':'carrot'},
    {'Animal':'crocodile', 'Food': 'lecturer'}
)

animal_food_t = (
    ('rabbit','carrot'), 
    ('mouse','cheese'), 
    ('crocodile','student'),
    ('rabbit','lettuce'),
    ('guineapig','carrot'),
    ('crocodile', 'lecturer')
)

animal_foods_dict = {
    'mouse': {'cheese'},
    'crocodile': {'student', 'lecturer'},
    'rabbit': {'carrot', 'lettuce'},
    'guineapig': {'carrot'}
}

Animals = set(pluck('Animal', animal_food_dl))
Foods = set(pluck('Food', animal_food_dl))

Animals, Foods
Animals_t, Foods_t = tuple(sorted(Animals)), tuple(sorted(Foods))

AnimalsFoods = set.union(Animals, Foods)
AnimalsFoods_t = tuple(sorted(AnimalsFoods))

({'crocodile', 'guineapig', 'mouse', 'rabbit'},
 {'carrot', 'cheese', 'lecturer', 'lettuce', 'student'})

In [48]:
def rev(t):
    return tuple(reversed(t))

def seqToIndexMap(seq):
    sorted_seq = sorted(seq)
    myIndexMap = dict(map(rev, enumerate(sorted_seq)))
    return myIndexMap

def indexToSeqMap(seq):
    sorted_seq = sorted(seq)
    mySeqMap = dict(enumerate(sorted_seq))
    return mySeqMap

def seqMapToOneHots(seqMap):
    n = len(seqMap.keys())
    one_hots = np.zeros((n,n), dtype=np.uint8)
    for (seq, idx) in seqMap.items():
        one_hots[idx][idx] = 1.0
    return one_hots

def xToOneHot(x, seqMap, oneHots):
    return oneHots[seqMap[x]]

def seqToOneHotMap(seq):
    seqMap = seqToIndexMap(seq)
    oneHots = seqMapToOneHots(seqMap)
    return {seq:oneHots[seqMap[seq]]
            for seq in seqMap}

def oneHotToSeqMap(seq):
    sorted_seq = sorted(seq)
    seqToOHmap = seqToOneHotMap(seq)
    seqToOHmap_t = walk_values(tuple, seqToOHmap)
#     seqToOHmap_t = mapValues(tuple, seqToOHmap)
    OHtoSeq_dict = dict(map(rev, seqToOHmap_t.items()))
    def OHtoSeq(OH_vector):
        OH_t = tuple(OH_vector)
        return OHtoSeq_dict[OH_t]
    return OHtoSeq

In [51]:
print('Seq⟷Index')
seqToIndexMap(Animals_t)
indexToSeqMap(Animals_t)

print('Seq⟷Local')
seqMapToOneHots(seqToIndexMap(Animals_t))
xToOneHot('mouse', seqToIndexMap(Animals_t), seqMapToOneHots(seqToIndexMap(Animals_t)))
animalToOH = partial(xToOneHot, seqMap = seqToIndexMap(Animals_t), oneHots = seqMapToOneHots(seqToIndexMap(Animals_t)))
animalToOH('mouse')

seqToOneHotMap(Animals_t)
animalToOH_dict = seqToOneHotMap(Animals_t)
animalToOH_dict['mouse']
oneHotToSeqMap(Animals_t)(animalToOH_dict['mouse'])

Seq⟷Index


{'crocodile': 0, 'guineapig': 1, 'mouse': 2, 'rabbit': 3}

{0: 'crocodile', 1: 'guineapig', 2: 'mouse', 3: 'rabbit'}

Seq⟷Local


array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]], dtype=uint8)

array([0, 0, 1, 0], dtype=uint8)

array([0, 0, 1, 0], dtype=uint8)

{'crocodile': array([1, 0, 0, 0], dtype=uint8),
 'guineapig': array([0, 1, 0, 0], dtype=uint8),
 'mouse': array([0, 0, 1, 0], dtype=uint8),
 'rabbit': array([0, 0, 0, 1], dtype=uint8)}

array([0, 0, 1, 0], dtype=uint8)

'mouse'

In [42]:
def local(activation_vector):
    v = activation_vector
    return v.sum() == 1 and len(v.nonzero()[0]) == 1

seqToOneHotMap(Animals_t)['mouse']
local(seqToOneHotMap(Animals_t)['mouse'])
local(np.array([0, 1, 1]))
local(np.array([0, 1, -1]))
local(np.array([0, 0.5, -0.5]))

array([0, 0, 1, 0], dtype=uint8)

True

False

False

False

# Sets / Boolean Algebra

In [44]:
Animals

{'crocodile', 'guineapig', 'mouse', 'rabbit'}

In [63]:
In_The_Yard = {'mouse', 'rabbit'}

{a:animalToOH(a) for a in In_The_Yard}

In_The_Yard_OH = tuple(lmap(animalToOH, In_The_Yard))
In_The_Yard_OH

{'mouse': array([0, 0, 1, 0], dtype=uint8),
 'rabbit': array([0, 0, 0, 1], dtype=uint8)}

(array([0, 0, 1, 0], dtype=uint8), array([0, 0, 0, 1], dtype=uint8))

In [46]:
def tpr_union(u, v):
    return u + v

In [82]:
In_The_Yard_tpr = reduce(tpr_union, In_The_Yard_OH)
In_The_Yard_tpr

np.array_equal( In_The_Yard_tpr, tpr_union(animalToOH('mouse'),
                                           animalToOH('rabbit')) )

array([0, 0, 1, 1], dtype=uint8)

True

In [133]:
def tpr_set_to_indices(S):
    return S.nonzero()[0]

tpr_set_to_indices(In_The_Yard_tpr)

def tpr_set_to_labels(S, idx_to_seq):
    indices = tpr_set_to_indices(S)
    seqs = {idx_to_seq[idx] for idx in indices}
    return seqs


def Animal_set(animal_set_tpr):
    return tpr_set_to_labels(animal_set_tpr, indexToSeqMap(Animals_t))

tpr_set_to_labels(In_The_Yard_tpr, indexToSeqMap(Animals_t))
Animal_set(In_The_Yard_tpr)


def Food_set(food_set_tpr):
    return tpr_set_to_labels(food_set_tpr, indexToSeqMap(Foods_t))

def AnimalsFoods_set(AnimalsFoods_set_tpr):
    return tpr_set_to_labels(AnimalsFoods_set_tpr, indexToSeqMap(AnimalsFoods_t))

array([2, 3])

{'mouse', 'rabbit'}

{'mouse', 'rabbit'}

In [78]:
def tpr_intersection(u, v):
    return u * v

tpr_intersection(In_The_Yard_tpr, animalToOH('mouse'))
Animal_set(tpr_intersection(In_The_Yard_tpr, animalToOH('mouse')))

tpr_intersection(In_The_Yard_tpr, animalToOH('crocodile'))
Animal_set(tpr_intersection(In_The_Yard_tpr, animalToOH('crocodile')))

array([0, 0, 1, 0], dtype=uint8)

{'mouse'}

array([0, 0, 0, 0], dtype=uint8)

set()

In [79]:
def tpr_subtraction(u,v):
    return u - v

In_The_Yard_tpr
tpr_subtraction(In_The_Yard_tpr, animalToOH('mouse'))
Animal_set(tpr_subtraction(In_The_Yard_tpr, animalToOH('mouse')))
animalToOH('mouse')
animalToOH('rabbit')

array([0, 0, 1, 1], dtype=uint8)

array([0, 0, 0, 1], dtype=uint8)

{'rabbit'}

array([0, 0, 1, 0], dtype=uint8)

array([0, 0, 0, 1], dtype=uint8)

In [80]:
def tpr_complement(u):
    return np.ones(len(u), dtype=np.uint8) - u

In_The_Yard_tpr
tpr_complement(In_The_Yard_tpr)
Animal_set(tpr_complement(In_The_Yard_tpr))

array([0, 0, 1, 1], dtype=uint8)

array([1, 1, 0, 0], dtype=uint8)

{'crocodile', 'guineapig'}

# (...multi-valued) functions ("Filler/role bindings")

In [87]:
Roles = set(animal_food_dl[0].keys())
Roles

Roles_t = tuple(sorted(Roles))

roleToOH = partial(xToOneHot, seqMap = seqToIndexMap(Roles_t), oneHots = seqMapToOneHots(seqToIndexMap(Roles_t)))
roleToOH_dict = seqToOneHotMap(Roles_t)
roleToOH_dict

{'Animal', 'Food'}

{'Animal': array([1, 0], dtype=uint8), 'Food': array([0, 1], dtype=uint8)}

In [106]:
foodToOH = partial(xToOneHot, seqMap = seqToIndexMap(Foods_t), oneHots = seqMapToOneHots(seqToIndexMap(Foods_t)))
foodToOH_dict = seqToOneHotMap(Foods_t)
foodToOH_dict

{'carrot': array([1, 0, 0, 0, 0], dtype=uint8),
 'cheese': array([0, 1, 0, 0, 0], dtype=uint8),
 'lecturer': array([0, 0, 1, 0, 0], dtype=uint8),
 'lettuce': array([0, 0, 0, 1, 0], dtype=uint8),
 'student': array([0, 0, 0, 0, 1], dtype=uint8)}

In [123]:
animal_food_dl[0]
animal_food_dl[3]

{'Animal': 'rabbit', 'Food': 'carrot'}

{'Animal': 'rabbit', 'Food': 'lettuce'}

## Storage

In [119]:
print('rabbit/Animal', '')
animalToOH_dict['rabbit'] #filler
roleToOH_dict['Animal'] #role = variable

# fv.T corresponding to "rabbit/Animal"
np.matmul(animalToOH_dict['rabbit'][:, None], roleToOH_dict['Animal'][:, None].T)
np.outer(animalToOH_dict['rabbit'], roleToOH_dict['Animal'])

rabbit_Animal = np.outer(animalToOH_dict['rabbit'], roleToOH_dict['Animal'])

rabbit/Animal 


array([0, 0, 0, 1], dtype=uint8)

array([1, 0], dtype=uint8)

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 0]], dtype=uint8)

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 0]], dtype=uint8)

In [138]:
def bind_filler_to_role(filler_vector, role_vector):
    return np.outer(filler_vector, role_vector)

bind_filler_to_role(animalToOH_dict['rabbit'], roleToOH_dict['Animal'])

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 0]], dtype=uint8)

In [120]:
print('carrot+lettuce/Food', '')
foodToOH_dict['carrot']
foodToOH_dict['lettuce']
tpr_union(foodToOH_dict['carrot'], foodToOH_dict['lettuce'])
roleToOH_dict['Food']
np.matmul(tpr_union(foodToOH_dict['carrot'], foodToOH_dict['lettuce'])[:, None],
          roleToOH_dict['Food'][:, None].T)
np.outer(foodToOH_dict['carrot'] + foodToOH_dict['lettuce'],
         roleToOH_dict['Food'])

carrotLettuce_Food = np.outer(foodToOH_dict['carrot'] + foodToOH_dict['lettuce'],
                              roleToOH_dict['Food'])

carrot+lettuce/Food 


array([1, 0, 0, 0, 0], dtype=uint8)

array([0, 0, 0, 1, 0], dtype=uint8)

array([1, 0, 0, 1, 0], dtype=uint8)

array([0, 1], dtype=uint8)

array([[0, 1],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0]], dtype=uint8)

array([[0, 1],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0]], dtype=uint8)

In [130]:
animal_food_dl[0]
animal_food_dl[3]

animalFoodToOH = partial(xToOneHot, seqMap = seqToIndexMap(AnimalsFoods_t), oneHots = seqMapToOneHots(seqToIndexMap(AnimalsFoods_t)))
animalFoodToOH_dict = seqToOneHotMap(AnimalsFoods_t)
animalFoodToOH_dict

rabbit_Animal_full = np.outer(animalFoodToOH_dict['rabbit'], roleToOH_dict['Animal'])
rabbit_Animal_full

carrotLettuce_Food_full = np.outer(animalFoodToOH_dict['carrot'] + animalFoodToOH_dict['lettuce'],
                                   roleToOH_dict['Food'])
carrotLettuce_Food_full

rabbit_Animal__carrotLettuce_Food = rabbit_Animal_full + carrotLettuce_Food_full
rabbit_Animal__carrotLettuce_Food

{'Animal': 'rabbit', 'Food': 'carrot'}

{'Animal': 'rabbit', 'Food': 'lettuce'}

{'carrot': array([1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'cheese': array([0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'crocodile': array([0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=uint8),
 'guineapig': array([0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=uint8),
 'lecturer': array([0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8),
 'lettuce': array([0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8),
 'mouse': array([0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8),
 'rabbit': array([0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8),
 'student': array([0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)}

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0]], dtype=uint8)

array([[0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=uint8)

array([[0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 0],
       [0, 0]], dtype=uint8)

## Retrieval

In [135]:
#retrieve Animal value
np.dot(rabbit_Animal__carrotLettuce_Food, 
       roleToOH_dict['Animal'])
AnimalsFoods_set(np.dot(rabbit_Animal__carrotLettuce_Food, roleToOH_dict['Animal']))

#retrieve Food value
np.dot(rabbit_Animal__carrotLettuce_Food, 
       roleToOH_dict['Food'])
AnimalsFoods_set(np.dot(rabbit_Animal__carrotLettuce_Food, roleToOH_dict['Food']))

array([0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8)

{'rabbit'}

array([1, 0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8)

{'carrot', 'lettuce'}

In [136]:
def retrieve_filler(binding_matrix, role_vector):
    return np.dot(binding_matrix, role_vector)

retrieve_filler(rabbit_Animal__carrotLettuce_Food,
                roleToOH_dict['Animal'])

array([0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8)

In [137]:
def role_has_filler(binding_matrix, role_vector, filler_vector):
    return np.dot(retrieve_filler(binding_matrix, role_vector), filler_vector)

role_has_filler(rabbit_Animal__carrotLettuce_Food,
                roleToOH_dict['Animal'],
                animalFoodToOH_dict['rabbit'])

1

In [140]:
animal_foods_dict

({'mouse': {'cheese'}},
 {'crocodile': {'lecturer', 'student'}},
 {'rabbit': {'carrot', 'lettuce'}},
 {'guineapig': {'carrot'}})

In [141]:
Animal_Foods = np.zeros((len(Foods), len(Animals)), dtype=np.uint8)
Animal_Foods

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=uint8)

In [ ]:
In_The_Yard_OH = tuple(lmap(animalToOH, In_The_Yard))
In_The_Yard_tpr = reduce(tpr_union, In_The_Yard_OH)

In [143]:
animal_foods_dict

({'mouse': {'cheese'}},
 {'crocodile': {'lecturer', 'student'}},
 {'rabbit': {'carrot', 'lettuce'}},
 {'guineapig': {'carrot'}})

In [183]:
for animal in animal_foods_dict:
    foods = animal_foods_dict[animal]
    
    role = animal
    filler = foods
    
    filler_OHs = tuple(lmap(foodToOH, foods))
    filler_tpr = reduce(tpr_union, filler_OHs)
    
    filler_vector = filler_tpr
#     filler_vector = foodToOH_dict[filler]
    role_vector = animalToOH_dict[role]
    
    Animal_Foods += bind_filler_to_role(filler_vector, role_vector)

del foods
del role
del filler
del filler_OHs
del filler_tpr
del filler_vector
del role_vector

Animal_Foods

array([[0, 2, 0, 2],
       [0, 0, 2, 0],
       [2, 0, 0, 0],
       [0, 0, 0, 2],
       [2, 0, 0, 0]], dtype=uint8)

In [184]:
retrieve_filler(Animal_Foods, 
                animalToOH_dict['rabbit'])

Food_set(retrieve_filler(Animal_Foods, animalToOH_dict['rabbit']))

array([2, 0, 0, 2, 0], dtype=uint8)

{'carrot', 'lettuce'}

In [185]:
def retrieve_roles_with_filler_index(binding_matrix, filler_index):
    role_indices = binding_matrix[filler_index].nonzero()[0]
    n_roles = Animal_Foods.shape[1]
    role_tpr = np.zeros(n_roles, dtype = np.uint8)
    np.put(role_tpr, role_indices, 1)
    return role_tpr

foodToOH_dict['carrot']
foodToOH_dict['carrot'].nonzero()[0].item()
retrieve_roles_with_filler_index(Animal_Foods,
                                 foodToOH_dict['carrot'].nonzero()[0].item())
Animal_set(retrieve_roles_with_filler_index(Animal_Foods,
                                            foodToOH_dict['carrot'].nonzero()[0].item()))

array([1, 0, 0, 0, 0], dtype=uint8)

0

array([0, 1, 0, 1], dtype=uint8)

{'guineapig', 'rabbit'}

# Distributed Representations

In [174]:
from scipy.linalg import hadamard

In [187]:
def next_power_of_2(x):
    power = 1;
    while(power < x):
        power *= 2
    return power

def distributed_basis(S):
    n = len(S)
    N = next_power_of_2(n)
    return np.sqrt(N) * hadamard(N)

distributed_basis(Foods)

array([[ 2.82842712,  2.82842712,  2.82842712,  2.82842712,  2.82842712,
         2.82842712,  2.82842712,  2.82842712],
       [ 2.82842712, -2.82842712,  2.82842712, -2.82842712,  2.82842712,
        -2.82842712,  2.82842712, -2.82842712],
       [ 2.82842712,  2.82842712, -2.82842712, -2.82842712,  2.82842712,
         2.82842712, -2.82842712, -2.82842712],
       [ 2.82842712, -2.82842712, -2.82842712,  2.82842712,  2.82842712,
        -2.82842712, -2.82842712,  2.82842712],
       [ 2.82842712,  2.82842712,  2.82842712,  2.82842712, -2.82842712,
        -2.82842712, -2.82842712, -2.82842712],
       [ 2.82842712, -2.82842712,  2.82842712, -2.82842712, -2.82842712,
         2.82842712, -2.82842712,  2.82842712],
       [ 2.82842712,  2.82842712, -2.82842712, -2.82842712, -2.82842712,
        -2.82842712,  2.82842712,  2.82842712],
       [ 2.82842712, -2.82842712, -2.82842712,  2.82842712, -2.82842712,
         2.82842712,  2.82842712, -2.82842712]])

In [194]:
carrot_rabbit__mouse_cat = {
    'rabbit':{'carrot'},
    'cat':{'mouse'}
}
carrot_rabbit__mouse_cat

CarrotRabbitMouseCat = {'carrot', 'rabbit', 'mouse', 'cat'}
CarrotRabbitMouseCat_t = ('rabbit', 'mouse', 'carrot', 'cat') #following along with https://www.cse.unsw.edu.au/~billw/cs9444/tensor-stuff/tensor-intro-04.html
CarrotRabbitMouseCat_b = distributed_basis(CarrotRabbitMouseCat_t) / 4
CarrotRabbitMouseCat_b

{'rabbit': {'carrot'}, 'cat': {'mouse'}}

array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5]])

In [206]:
CarrotRabbitMouseCat_b[CarrotRabbitMouseCat_t.index('carrot')]
CarrotRabbitMouseCat_b[CarrotRabbitMouseCat_t.index('rabbit')]
carrot_rabbit = bind_filler_to_role(CarrotRabbitMouseCat_b[CarrotRabbitMouseCat_t.index('carrot')],
                                    CarrotRabbitMouseCat_b[CarrotRabbitMouseCat_t.index('rabbit')])
(4 * carrot_rabbit).astype(np.int8)

retrieve_filler(carrot_rabbit, CarrotRabbitMouseCat_b[CarrotRabbitMouseCat_t.index('rabbit')])
CarrotRabbitMouseCat_b[CarrotRabbitMouseCat_t.index('carrot')]

array([ 0.5,  0.5, -0.5, -0.5])

array([0.5, 0.5, 0.5, 0.5])

array([[ 1,  1,  1,  1],
       [ 1,  1,  1,  1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1]], dtype=int8)

array([ 0.5,  0.5, -0.5, -0.5])

array([ 0.5,  0.5, -0.5, -0.5])